In [1]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score, median_absolute_error
from sklearn.ensemble import RandomForestRegressor
from scipy import stats

In [4]:
df_modelo = pd.read_csv('df_eda_limpio.csv')
df_modelo.head()

,make,model,version,fuel,year,kms,power,shift,price,dealer_name,...,province,vehicle_age,price_per_power,price_per_year,dealer_info,power_per_kms,make_popularity,model_popularity,big_city_dealer,price_range
0,Opel,Crossland,1.2 GAS 110 GS Line 5p S/S,Gasolina,2022,5.0,110.0,manual,22900,Sergio Y.,...,Barcelona,3,208.19,7633.34,"Sergio Y.\r\n2, Carrer de Jacint Benavente, Po...",22.000000,4591,158,0,20-30k
1,Opel,Crossland,1.2 81kW (110CV) GS Line,Gasolina,2022,24847.0,110.0,manual,19990,Peugeot Alcala 534,...,Madrid,3,181.73,6663.34,"Peugeot Alcala 534\r\nAvenida de José Gárate, ...",0.004427,4591,158,0,10-20k
2,Opel,Crossland,1.5D 88kW (120CV) Business Elegance Auto,Diésel,2021,41356.0,120.0,automatic,18590,Clicars S.,...,Madrid,4,154.92,4647.50,"Clicars S.\r\nSan Cristóbal, Avenida de Andalu...",0.002902,4591,158,1,10-20k
3,Opel,Crossland,GS-Line 1.2 GAS MT6 S/S 110cv,Gasolina,2022,11.0,110.0,manual,22700,Vallescar S.,...,Barcelona,3,206.37,7566.67,"Vallescar S.\r\nParc de Bombers de Sabadell, 5...",10.000000,4591,158,0,20-30k
4,Opel,Crossland,GS-Line 1.2 GAS MT6 S/S 110cv,Gasolina,2022,11.0,110.0,manual,22700,Vallescar Ocasion M.,...,Barcelona,3,206.37,7566.67,"Vallescar Ocasion M.\r\n27, Carrer de Fèlix Fe...",10.000000,4591,158,0,20-30k


In [11]:
# Características originales
features = ['year', 'kms', 'power', 'vehicle_age', 'fuel', 'shift', 'make', 'model']
target = 'price'

# Separamos en variables originales
X = df_modelo[features].copy()
y = df_modelo[target]

In [12]:
# Creamos nuevas características que pueden aportar valor al modelo
# Potencia relativa a la edad (los coches potentes pierden valor más rápido con la edad)
X['power_per_age'] = df_modelo['power'] / (df_modelo['vehicle_age'] + 1)

# Kilometraje anual (un indicador importante del uso del vehículo)
X['km_per_year'] = df_modelo['kms'] / (df_modelo['vehicle_age'] + 1)

# Transformación logarítmica de kilometraje (suaviza valores extremos)
X['log_kms'] = np.log1p(df_modelo['kms'])

# Mostramos las primeras filas de nuestro dataset mejorado
X.head()

,year,kms,power,vehicle_age,fuel,shift,make,model,power_per_age,km_per_year,log_kms
0,2022,5.0,110.0,3,Gasolina,manual,Opel,Crossland,27.5,1.25,1.791759
1,2022,24847.0,110.0,3,Gasolina,manual,Opel,Crossland,27.5,6211.75,10.120533
2,2021,41356.0,120.0,4,Diésel,automatic,Opel,Crossland,24.0,8271.20,10.629997
3,2022,11.0,110.0,3,Gasolina,manual,Opel,Crossland,27.5,2.75,2.484907
4,2022,11.0,110.0,3,Gasolina,manual,Opel,Crossland,27.5,2.75,2.484907


In [13]:
# Definimos cuáles son las características numéricas, incluyendo las nuevas
numeric_features = ['year', 'kms', 'power', 'vehicle_age']
numeric_features_extended = numeric_features + ['power_per_age', 'km_per_year', 'log_kms']
categorical_features = ['fuel', 'shift', 'make', 'model']

# Detectamos outliers mediante Z-score
# Calculamos los Z-scores para características numéricas
z_scores = stats.zscore(X[numeric_features_extended])
abs_z_scores = np.abs(z_scores)

# Consideramos outliers valores con Z-score > 3 (muy alejados de la media)
filtered_entries = (abs_z_scores < 3).all(axis=1)

# Filtramos el dataset para eliminar los outliers
X_filtered = X[filtered_entries]
y_filtered = y[filtered_entries]

# Comprobamos cuántos registros se eliminaron
print(f"Registros originales: {len(X)}")
print(f"Registros después de eliminar outliers: {len(X_filtered)}")
print(f"Registros eliminados: {len(X) - len(X_filtered)} ({(len(X) - len(X_filtered))/len(X)*100:.2f}%)")

Registros originales: 91040
Registros después de eliminar outliers: 86774
Registros eliminados: 4266 (4.69%)


In [14]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, test_size=0.2, random_state=42
)

# Verificamos las dimensiones de los conjuntos
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (69419, 11)
X_test shape: (17355, 11)
y_train shape: (69419,)
y_test shape: (17355,)


In [15]:
# Definimos el preprocesador para las características numéricas y categóricas
preprocessor = ColumnTransformer(
    transformers=[
        # Estandarizamos variables numéricas (media 0, desviación 1)
        ('num', StandardScaler(), numeric_features_extended),
        # Codificamos variables categóricas con one-hot encoding
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Este preprocesador se aplicará de forma automática en el pipeline

In [16]:
# Definimos varios modelos avanzados con configuraciones iniciales
# RandomForest con mayor número de árboles
rf_model = RandomForestRegressor(
    n_estimators=300,  # Más árboles para mejor generalización
    max_depth=None,    # Permitimos árboles profundos
    min_samples_split=2,
    random_state=42
)

In [17]:
# Actualizamos el diccionario de modelos
models = {
    'RandomForest': rf_model
}

In [18]:
# Configuramos la validación cruzada con 5 folds
cv_results = {}
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Para cada modelo, ejecutamos validación cruzada y guardamos los resultados
for name, model in models.items():
    print(f"Entrenando modelo: {name}")
    
    # Creamos un pipeline que incluye preprocesamiento y modelo
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    
    # Almacenamos los scores de cada fold
    scores = []
    
    # Realizamos la validación cruzada
    for fold, (train_idx, test_idx) in enumerate(kf.split(X_train)):
        # Separamos los datos para este fold
        X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_cv, y_val_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]
        
        # Entrenamos el pipeline
        pipeline.fit(X_train_cv, y_train_cv)
        
        # Evaluamos en el conjunto de validación
        y_pred_cv = pipeline.predict(X_val_cv)
        fold_score = r2_score(y_val_cv, y_pred_cv)
        scores.append(fold_score)
        
        print(f"  Fold {fold+1}: R² = {fold_score:.4f}")
    
    # Calculamos estadísticas de los scores
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    
    print(f"  Media R²: {mean_score:.4f} (±{std_score:.4f})")
    
    # Entrenamos el modelo con todos los datos de entrenamiento
    pipeline.fit(X_train, y_train)
    
    # Guardamos los resultados
    cv_results[name] = {
        'mean_r2': mean_score,
        'std_r2': std_score,
        'pipeline': pipeline
    }
    
    print("-" * 50)

Entrenando modelo: RandomForest
  Fold 1: R² = 0.9677
  Fold 2: R² = 0.9705
  Fold 3: R² = 0.9696
  Fold 4: R² = 0.9746
  Fold 5: R² = 0.9748
  Media R²: 0.9715 (±0.0028)
--------------------------------------------------


In [19]:
# Evaluamos cada modelo individualmente en el conjunto de prueba
individual_results = {}

for name, result in cv_results.items():
    pipeline = result['pipeline']
    y_pred = pipeline.predict(X_test)
    
    # Calculamos métricas de rendimiento
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    individual_results[name] = {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'r2': r2,
        'predictions': y_pred
    }
    
    print(f"Resultados para {name}:")
    print(f"  RMSE: {rmse:.2f} €")
    print(f"  MAE: {mae:.2f} €")
    print(f"  MAPE: {mape:.4f} %")
    print(f"  R²: {r2:.4f}")
    print("-" * 50)

Resultados para RandomForest:
  RMSE: 1311.00 €
  MAE: 296.46 €
  MAPE: 0.2045 %
  R²: 0.9791
--------------------------------------------------


In [6]:
import re

In [7]:
# Eliminar los caballos de la columna 'version'
df_modelo['version'] = df_modelo['version'].apply(lambda x: re.sub(r'\s*\(\d+\s*CV\)', '', x))

In [8]:
#Se guarda el fichero con el mejor modelo 

df_modelo.to_csv("modelos_coches.csv", index=False)

In [20]:
from joblib import dump

In [24]:
import os

In [22]:
# Obtener el pipeline de RandomForest (mejor modelo)
mejor_modelo_pipeline = cv_results['RandomForest']['pipeline']

In [25]:
# Guardar con joblib (a menudo más eficiente para modelos de scikit-learn)
dump(mejor_modelo_pipeline, 'modelo_coches_rf.joblib', compress=3)

# Verificar tamaño
tamano_archivo_mb = os.path.getsize('modelo_coches_rf.joblib') / (1024 * 1024)
print(f"Tamaño con joblib: {tamano_archivo_mb:.2f} MB")

Tamaño con joblib: 80.48 MB
